# Derivatives, basic NN

In [1]:
import numpy as np

from lib.value import Value
from lib.linear_algebra import Vector, Matrix
from lib.nn import Linear, Sigmoid, NN

np.random.seed(1)

### Derivatives

In [2]:
X = 5
W = 2
b = 0
y = 12

In [3]:
Z = X*W + b
L = (Z - y)**2

dZ_dL = 2*Z - 2*y 
dW_dZ = X
dW_dL = dZ_dL * dW_dZ
db_dZ = 1
db_dL = dZ_dL * db_dZ
L, (Z,dZ_dL), (W, dW_dL), (b, db_dL)

(4, (10, -4), (2, -20), (0, -4))

In [4]:
X = Value(5)
W = Value(2)
b = Value(0)

Z = X * W + b

y = Value(12)

L = (Z - y) ** 2
L.grad = 1
L.backward()
L, Z, W, b

({b5b53cf5, 4, 1}, {226597e0, 10, -4}, {e841215c, 2, -20}, {5e194b9b, 0, -4})

### Linear operations

In [5]:
m1 = Matrix([
    [1, 4, 1],
    [2, 3, 0]
])

m2 = Matrix([
    [1, -2, 4, 4],
    [3, 4, 6, 1],
    [5, 6, 2, 1]
])

v1 = Vector([2, 3, 4, 5]) 

print(m1.matmul(m2) + v1)

Matrix([
[{ef89d613, 20, 0}, {262f4b12, 23, 0}, {66615823, 34, 0}, {97fba7d2, 14, 0}],
[{afda3e06, 13, 0}, {2d2e4633, 11, 0}, {c0f0bf0c, 30, 0}, {e6e9485c, 16, 0}]
])


# NN

In [6]:
X = Matrix(np.random.uniform(size=(10, 5)))
y = Matrix([[1] for _ in range(10)])

nn = NN([
    Linear(5, 3),
    Sigmoid(),
    Linear(3, 1),
])

In [7]:
for i in range(30):
    for p in nn.params():
        p.zero_grad()

    out = nn(X)
    L = sum([i[0] for i in (y - out) ** 2]) / y.dims()[0]
    print(f"{i} {L.data:.2f}")
    L.grad = 1
    L.backward()

    for p in nn.params():
        for v in p.all_values():
            v.data -= 0.1 * v.grad


0 0.29
1 0.17
2 0.12
3 0.11
4 0.10
5 0.10
6 0.10
7 0.09
8 0.09
9 0.09
10 0.09
11 0.09
12 0.08
13 0.08
14 0.08
15 0.08
16 0.08
17 0.08
18 0.07
19 0.07
20 0.07
21 0.07
22 0.07
23 0.07
24 0.07
25 0.06
26 0.06
27 0.06
28 0.06
29 0.06


In [8]:
out, y

(Matrix([
 [{828ddab7, 0.82, -0.04}],
 [{6cc8fb46, 1.1, 0.02}],
 [{20756a79, 0.85, -0.03}],
 [{23531f2c, 1.08, 0.02}],
 [{a94d7f5e, 0.83, -0.03}],
 [{bef753df, 0.69, -0.06}],
 [{736c2e10, 1.44, 0.09}],
 [{84a00368, 1.37, 0.07}],
 [{982d3998, 0.74, -0.05}],
 [{8d5d1d63, 0.99, -0.0}]
 ]),
 Matrix([
 [{2e3803c9, 1, 1.06}],
 [{b29813b9, 1, -0.99}],
 [{d18a07de, 1, 0.94}],
 [{2e648e5c, 1, -0.8}],
 [{5c9ec651, 1, 1.02}],
 [{75e5d1f7, 1, 1.81}],
 [{b0a57a36, 1, -3.26}],
 [{5772a432, 1, -2.82}],
 [{f422a23f, 1, 1.67}],
 [{9a2a75fc, 1, -0.07}]
 ]))